In [2]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
import seaborn as sns
import sqlite3

In [7]:
conection = sqlite3.connect("amazon_sales.db")
df = pd.read_sql_query("""
                        SELECT *
                        FROM products
                        JOIN (
                            SELECT reviews.review_title,reviews.review_content, users.user_name,reviews.id_product
                            FROM reviews
                            JOIN users ON reviews.id_user = users.id
                        ) AS reviews_users ON products.id = reviews_users.id_product; 
                        """,conection)

In [8]:
df.head()

,id,product_name,category_1,category_2,category_3,category_4,category_5,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,img_link,product_link,review_title,review_content,user_name,id_product
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Satisfied,Looks durable Charging is fine tooNo complains,Manav,B07JW9H4J1
1,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Charging is really fast,Charging is really fast,Adarsh gupta,B07JW9H4J1
2,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Value for money,good product.,Sundeep,B07JW9H4J1
3,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Product review,Till now satisfied with the quality.,S.Sayeed Ahmed,B07JW9H4J1
4,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Good quality,This is a good product . The charging speed is...,jaspreet singh,B07JW9H4J1


In [13]:
df.shape

(2590, 19)

In [10]:
numeric_df = df[["discounted_price",	"actual_price",	"discount_percentage",	"rating",	"rating_count"]]

In [11]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590 entries, 0 to 2589
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   discounted_price     2590 non-null   float64
 1   actual_price         2590 non-null   float64
 2   discount_percentage  2590 non-null   float64
 3   rating               2590 non-null   float64
 4   rating_count         2590 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 101.3 KB


# Handeling Missing Values

In [12]:
df.isna().sum()

id                     0
product_name           0
category_1             0
category_2             0
category_3             0
category_4             0
category_5             0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
img_link               0
product_link           0
review_title           0
review_content         0
user_name              0
id_product             0
dtype: int64

# Handeling Duplicates

In [18]:
df[df.duplicated()]

,id,product_name,category_1,category_2,category_3,category_4,category_5,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,img_link,product_link,review_title,review_content,user_name,id_product


# Preguntas Frecuentes 

1. ¿Cuáles son los productos más vendidos?
2. ¿Existen patrones estacionales en las ventas?
3. ¿Qué canales de marketing son más efectivos?
4. ¿Cuál es el valor promedio de un pedido?
5. ¿Qué segmentos de clientes son más rentables?

In [ ]:
#asdfs